En résumé ultra-court

Word2Vec encode chaque mot en vecteur dense.On entraîne un seul modèle Word2Vec sur toutes les paires (mot1, mot2) de ton fichier base_relations.csv.

Pour chaque couple (mot1, mot2), on crée un vecteur combiné [v1, v2, |v1−v2|, v1*v2].

On entraîne un arbre de décision pour classifier ce vecteur en une relation.

Pour un nouveau couple, on encode + combine → on prédit la relation avec l’arbre.

In [ ]:
import re
import pandas as pd
import unicodedata
from pathlib import Path

# Variantes de "de"
PREPOSITIONS = [
    r"\bde la\b",
    r"\bde\b",
    r"\bdu\b",
    r"\bdes\b",
    r"\bd’\b",      # apostrophe typographique
    r"\bd'\b",      # apostrophe simple
    r"\bd’un\b",
    r"\bd'une\b",
    r"\bde l’\b",
    r"\bde l'\b",
]

# Pattern pour "N de N" (avec éventuellement un champ après un "|", qu'on ignore ici)
pattern = re.compile(
    rf"(.+?)\s+(?:{'|'.join(PREPOSITIONS)})\s+(.+?)(?:\s*\|\s*(.*))?$",
    flags=re.IGNORECASE
)


def nettoyer_texte(s: str) -> str:
    """Nettoyage de base : trim, minuscules, accents, ponctuation, espaces multiples."""
    if s is None:
        return None
    s = s.strip().lower()

    # Normaliser les accents
    s = unicodedata.normalize('NFKD', s)
    s = ''.join(c for c in s if not unicodedata.combining(c))

    # Retirer ponctuation
    s = re.sub(r"[^\w\s]", " ", s)
    # Retirer chiffres (si non pertinents)
    s = re.sub(r"\d+", "", s)
    # Espaces multiples → un seul
    s = re.sub(r"\s+", " ", s).strip()
    return s


def extraire_paires(ligne: str, relation_label: str):
    """
    Extrait (mot1, mot2) à partir d'une ligne de type `N de N ...`
    et assigne la relation passée en argument (un seul type de relation par fichier).
    """
    m = pattern.match(ligne)
    if not m:
        return None
    mot1_raw, mot2_raw, _ = m.groups()

    mot1 = nettoyer_texte(mot1_raw)
    mot2 = nettoyer_texte(mot2_raw)

    if not mot1 or not mot2:
        return None

    return mot1, mot2, relation_label


def traiter_fichier(input_path: str, relation_label: str):
    """
    Lit un fichier texte, extrait toutes les paires (mot1, mot2, relation_label)
    et renvoie une liste de dict pour ce fichier.
    """
    resultats = []
    with open(input_path, encoding="utf-8") as f:
        for lineno, ligne in enumerate(f, start=1):
            ligne = ligne.strip()
            if not ligne:
                continue
            paire = extraire_paires(ligne, relation_label)
            if paire:
                mot1, mot2, relation = paire
                resultats.append({"mot1": mot1, "mot2": mot2, "relation": relation})
            else:
                # Tu peux décommenter si tu veux voir les lignes non reconnues
                # print(f"Ligne {lineno} non extraite dans {input_path}: {ligne}")
                pass
    return resultats


def traiter_dossier(dossier_txt: str,
                    output_csv_global: str,
                    creer_csv_par_fichier: bool = False):
    """
    Parcourt tous les *.txt du dossier, utilise le nom de fichier comme label de relation
    (ex. r_loc_in.txt -> relation = 'r_loc_in'), agrège tout dans un CSV global.
    Optionnel : un CSV par fichier.
    """
    toutes_paires = []

    for chemin in Path(dossier_txt).glob("*.txt"):
        relation_label = chemin.stem  # ex. 'r_loc_in', 'r_depict', etc.

        paires = traiter_fichier(str(chemin), relation_label)
        print(f"{chemin.name}: {len(paires)} paires extraites.")

        if creer_csv_par_fichier:
            df_f = pd.DataFrame(paires)
            df_f = df_f.drop_duplicates(subset=["mot1", "mot2", "relation"]).reset_index(drop=True)
            df_f.to_csv(f"{chemin.stem}.csv", index=False, encoding="utf-8")

        toutes_paires.extend(paires)

    # CSV global
    df_global = pd.DataFrame(toutes_paires)
    df_global = df_global.drop_duplicates(subset=["mot1", "mot2", "relation"]).reset_index(drop=True)
    df_global.to_csv(output_csv_global, index=False, encoding="utf-8")
    print(f"Total: {len(df_global)} paires uniques écrites dans {output_csv_global}")


if __name__ == "__main__":
    # À adapter : chemin du dossier où se trouvent tes r_depict.txt, r_has_causitif.txt, etc.
    DOSSIER_RELATIONS = "/content/sample_data/"  # <-- change ici
    SORTIE_GLOBALE = "base_relations.csv"

    traiter_dossier(
        dossier_txt=DOSSIER_RELATIONS,
        output_csv_global=SORTIE_GLOBALE,
        creer_csv_par_fichier=True  # mets False si tu ne veux QUE le CSV global
    )


r_depict.txt: 1045 paires extraites.
r_quantificateur.txt: 880 paires extraites.
r_processusinstr-1.txt: 678 paires extraites.
r_topic.txt: 996 paires extraites.
r_lieu_origine.txt: 438 paires extraites.
r_processusagent.txt: 870 paires extraites.
r_holo.txt: 868 paires extraites.
r_object_matière.txt: 1283 paires extraites.
r_has_causitif.txt: 961 paires extraites.
r_processuspatient.txt: 1207 paires extraites.
r_own-1.txt: 1074 paires extraites.
r_social_tie.txt: 803 paires extraites.
r_has_property.txt: 1021 paires extraites.
r_product_of.txt: 1188 paires extraites.
Total: 9885 paires uniques écrites dans base_relations.csv


In [ ]:
#!pip install gensim

In [ ]:
import pandas as pd
import numpy as np
from gensim.models import Word2Vec

df = pd.read_csv("base_relations.csv")  # colonnes: mot1, mot2, relation

# entraînement / chargement Word2Vec
emb_dim = 100
sentences = df[["mot1", "mot2"]].values.tolist()

w2v_model = Word2Vec(
    sentences=sentences,
    vector_size=emb_dim,
    window=5,
    min_count=1,
    workers=4,
    sg=1
)


In [ ]:
def get_vec(word, model, dim):
    if word in model.wv:
        return model.wv[word]
    else:
        return np.zeros(dim, dtype=np.float32)

def combine(v1, v2):
    v1 = np.asarray(v1, dtype=np.float32)
    v2 = np.asarray(v2, dtype=np.float32)
    diff = np.abs(v1 - v2)
    prod = v1 * v2
    return np.concatenate([v1, v2, diff, prod], axis=-1)  # dim = 4 * emb_dim


In [ ]:
from sklearn.preprocessing import LabelEncoder

X = []
for _, row in df.iterrows():
    v1 = get_vec(row["mot1"], w2v_model, emb_dim)
    v2 = get_vec(row["mot2"], w2v_model, emb_dim)
    X.append(combine(v1, v2))

X = np.stack(X)  # shape (N, 4*emb_dim)

le_rel = LabelEncoder()
y = le_rel.fit_transform(df["relation"])  # entiers 0..C-1


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, confusion_matrix

# split train / test (tu peux ajouter un val si besoin)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

# arbre de décision
tree_clf = DecisionTreeClassifier(
    max_depth=20,          # à ajuster
    min_samples_leaf=5,    # évite le sur-apprentissage
    class_weight="balanced",  # utile si les classes sont déséquilibrées
    random_state=42
)

tree_clf.fit(X_train, y_train)

# évaluation
y_pred = tree_clf.predict(X_test)
print(classification_report(y_test, y_pred, target_names=le_rel.classes_))
print(confusion_matrix(y_test, y_pred))


                    precision    recall  f1-score   support

          r_depict       0.00      0.00      0.00       195
    r_has_causitif       0.08      0.98      0.15       148
    r_has_property       0.00      0.00      0.00       164
            r_holo       0.00      0.00      0.00       138
    r_lieu_origine       0.33      0.03      0.05        73
  r_object_matière       0.83      0.37      0.51       120
           r_own-1       0.92      0.29      0.44        77
  r_processusagent       0.17      0.02      0.03       103
r_processusinstr-1       0.25      0.02      0.03       118
r_processuspatient       0.78      0.04      0.08       156
      r_product_of       0.97      0.16      0.28       210
  r_quantificateur       0.97      0.35      0.52       173
      r_social_tie       0.20      0.01      0.01       140
           r_topic       0.83      0.18      0.29       162

          accuracy                           0.18      1977
         macro avg       0.45      0.1

/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
def predict_relation_tree(mot1, mot2, model_w2v, emb_dim, tree_clf, label_encoder):
    v1 = get_vec(mot1, model_w2v, emb_dim)
    v2 = get_vec(mot2, model_w2v, emb_dim)
    v_pair = combine(v1, v2).reshape(1, -1)  # (1, 4*emb_dim)

    y_pred = tree_clf.predict(v_pair)[0]
    proba = tree_clf.predict_proba(v_pair)[0].max()  # probabilité de la classe prédite
    relation = label_encoder.inverse_transform([y_pred])[0]
    return relation, float(proba)

# Exemple d'utilisation
relation, proba = predict_relation_tree(
    "chien", "animal",
    w2v_model, emb_dim,
    tree_clf, le_rel
)
print("relation prédite :", relation, "confiance :", proba)


relation prédite : r_has_causitif confiance : 0.0810050555562672
